Instalasi & Import Library
Deskripsi:
- Langkah awal adalah menginstal dan mengimpor library yang dibutuhkan untuk mengambil, memproses, dan memvisualisasikan data spasial.

Fungsi Python:
1. pip install ... : Instalasi library dari terminal.
2. import ... : Mengimpor library ke script/notebook.

In [26]:
import requests
import folium
import geopandas as gpd
from shapely.geometry import shape
from pyproj import CRS
import json
import os


1️⃣ Ambil & Tampilkan Data Polygon dari API
Task:
- Ambil data polygon dari API MAPID dan tampilkan di peta interaktif.

Deskripsi:

1. Ambil data GeoJSON dari API.
2. Tampilkan seluruh polygon pada peta folium.

In [27]:
import requests, folium, geopandas as gpd
url = "https://geoserver.mapid.io/layers_new/get_layer?api_key=1f190186ad52452a9ab5d65b93ee48e1&layer_id=6988adc82d33abfe641f2132&project_id=69881afcbbaa47f3ce0ba819"
data = requests.get(url).json()
m = folium.Map(location=[-6.95, 107.68], zoom_start=12)
gj = folium.GeoJson(data, name="geojson").add_to(m)
m.fit_bounds(gj.get_bounds())
m


2️⃣ Konversi ke GeoDataFrame & Cek CRS
Task:
- Konversi data ke GeoDataFrame dan pastikan CRS sudah benar.

Deskripsi:

1. Konversi GeoJSON ke GeoDataFrame.
2. Set CRS ke EPSG:4326 jika belum ada.

In [28]:
print("CRS:", data.get("crs"))
print("Jumlah Features:", len(data.get("features", [])))

# 2. Identifikasi CRS dari nilai koordinat
features = data.get("features", [])
if features:
    coords = features[0].get("geometry", {}).get("coordinates", [])
    if coords:
        # Ambil sampel koordinat pertama
        if isinstance(coords[0], list):
            sampel = coords[0][0][0][:2] if isinstance(coords[0][0], list) else coords[0][:2]
        else:
            sampel = coords[:2]
        lon, lat = sampel
        
        # Identifikasi CRS
        if -180 <= lon <= 180 and -90 <= lat <= 90:
            detected_crs = "EPSG:4326"
            crs_desc = "WGS84 (derajat)"
        else:
            detected_crs = "EPSG:3857"
            crs_desc = "Projected (meter)"
        print(f"\nSampel Koordinat: lon={lon}, lat={lat}")
        print(f"Terdeteksi CRS: {detected_crs} ({crs_desc})")

# 3. Konversi GeoJSON ke GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(data['features'])

# 4. Set CRS jika belum ada
if gdf.crs is None:
    gdf = gdf.set_crs('EPSG:4326')
    print(f"\nCRS di-set ke EPSG:4326")
else:
    print(f"\nCRS sudah ada: {gdf.crs}")

# 5. Tampilkan info GeoDataFrame
print(f"\n=== Info GeoDataFrame ===")
print(f"Shape: {gdf.shape}")
print(f"CRS: {gdf.crs}")
print(f"Kolom: {gdf.columns.tolist()}")
print(f"\nData Preview:")
print(gdf.head())


CRS: None
Jumlah Features: 54

Sampel Koordinat: lon=110.81559643200006, lat=-7.576010058999941
Terdeteksi CRS: EPSG:4326 (WGS84 (derajat))

CRS di-set ke EPSG:4326

=== Info GeoDataFrame ===
Shape: (54, 28)
CRS: EPSG:4326
Kolom: ['geometry', 'fid', 'OBJECTID', 'ID_PROV', 'PROVINSI', 'ID_KABKOT', 'KABKOT', 'ID_KEC', 'KECAMATAN', 'ID_DESA', 'DESA', 'JUMLAH PENDUDUK', 'BELUM BEKERJA', 'SUDAH BEKERJA', 'TIDAK ATAU BELUM SEKOLAH', 'SD dan SMP', 'SLTA', 'D1 - D3', 'S1', 'S2 >', 'PDRB 2023', 'SKOR PDRB', 'SKOR PENDIDIKAN', 'SKOR PEKERJAAN', 'SKOR PRESENTASI RUMAH TANGGA TANPA LISTRIK', 'SKOR TOTAL', 'BOBOT AKHIR', 'SOCIOECONOMIC STATUS']

Data Preview:
                                            geometry   fid  OBJECTID  ID_PROV  \
0  MULTIPOLYGON Z (((110.8156 -7.57601 0, 110.815...  8288      8288       33   
1  MULTIPOLYGON Z (((110.81843 -7.56485 0, 110.81...  8287      8287       33   
2  MULTIPOLYGON Z (((110.83654 -7.5611 0, 110.836...  8286      8286       33   
3  MULTIPOLYGON Z (((

3️⃣ [Soal] Buffer Polygon
Deskripsi:
- Buat zona buffer (penyangga) sejauh 100 meter di sekitar setiap polygon pada data spasial. Buffer sering digunakan untuk analisis zona pengaruh atau area aman di sekitar objek spasial.

Soal:

1. Reproject data ke CRS meter (EPSG:3857).
2. Buat buffer 100 meter untuk setiap polygon.
3. Kembalikan ke EPSG:4326 dan export hasilnya ke GeoJSON.

Hint:

1. Gunakan .to_crs(), .buffer(), dan .to_file().

In [29]:
# 1. Reproject ke CRS meter (EPSG:3857)
gdf_proj = gdf.to_crs(epsg=3857)
print(f"Data di-reproject ke {gdf_proj.crs}")

# 2. Buat buffer 100 meter untuk setiap polygon
gdf_buffered = gdf_proj.copy()
gdf_buffered['geometry'] = gdf_proj.geometry.buffer(100)
print(f"Buffer 100 meter dibuat untuk {len(gdf_buffered)} polygon")

# 3. Kembalikan ke EPSG:4326
gdf_buffered = gdf_buffered.to_crs(epsg=4326)
print(f"Data di-reproject kembali ke {gdf_buffered.crs}")

# 4. Export ke GeoJSON
output_path = "ar_pop_by_zone_in_buffer.geojson"
gdf_buffered.to_file(output_path, driver='GeoJSON')
print(f"File tersimpan: {output_path}")

# 5. Tampilkan preview data buffered
print(f"\nPreview Buffer Polygon:")
print(gdf_buffered.head())


Data di-reproject ke EPSG:3857
Buffer 100 meter dibuat untuk 54 polygon
Data di-reproject kembali ke EPSG:4326
File tersimpan: ar_pop_by_zone_in_buffer.geojson

Preview Buffer Polygon:
                                            geometry   fid  OBJECTID  ID_PROV  \
0  POLYGON ((110.80531 -7.57795, 110.80506 -7.577...  8288      8288       33   
1  POLYGON ((110.81402 -7.56689, 110.81399 -7.566...  8287      8287       33   
2  POLYGON ((110.83114 -7.55832, 110.83112 -7.558...  8286      8286       33   
3  POLYGON ((110.7953 -7.54924, 110.79527 -7.5491...  8285      8285       33   
4  POLYGON ((110.83069 -7.56855, 110.83073 -7.568...  8284      8284       33   

      PROVINSI ID_KABKOT          KABKOT    ID_KEC   KECAMATAN        ID_DESA  \
0  JAWA TENGAH     33.72  KOTA SURAKARTA  33.72.02    SERENGAN  33.72.02.1004   
1  JAWA TENGAH     33.72  KOTA SURAKARTA  33.72.05  BANJARSARI  33.72.05.1007   
2  JAWA TENGAH     33.72  KOTA SURAKARTA  33.72.04      JEBRES  33.72.04.1009   
3  J

4️⃣ [Soal] Simplify Polygon
Deskripsi:
- Sederhanakan geometri polygon dengan toleransi 50 meter untuk mengurangi detail dan mempercepat visualisasi.

Soal:

1. Reproject ke EPSG:3857.
2. Simplify geometry dengan toleransi 50 meter.
3. Kembalikan ke EPSG:4326 dan export hasilnya ke GeoJSON.

Hint:
1. Gunakan .simplify() dan .to_file().

In [30]:
# 1. Reproject ke CRS meter (EPSG:3857)
gdf_proj = gdf.to_crs(epsg=3857)
print(f"Data di-reproject ke {gdf_proj.crs}")

# 2. Simplify geometry dengan toleransi 50 meter
gdf_simplified = gdf_proj.copy()
gdf_simplified['geometry'] = gdf_proj.geometry.simplify(50)
print(f"Simplify 50 meter diterapkan untuk {len(gdf_simplified)} polygon")

# 3. Kembalikan ke EPSG:4326
gdf_simplified = gdf_simplified.to_crs(epsg=4326)
print(f"Data di-reproject kembali ke {gdf_simplified.crs}")

# 4. Export ke GeoJSON
output_path = "ar_pop_by_zone_simplified.geojson"
gdf_simplified.to_file(output_path, driver='GeoJSON')
print(f"File tersimpan: {output_path}")

# 5. Tampilkan preview data simplified
print(f"\nPreview Simplified Polygon:")
print(gdf_simplified.head())


Data di-reproject ke EPSG:3857
Simplify 50 meter diterapkan untuk 54 polygon
Data di-reproject kembali ke EPSG:4326
File tersimpan: ar_pop_by_zone_simplified.geojson

Preview Simplified Polygon:
                                            geometry   fid  OBJECTID  ID_PROV  \
0  POLYGON Z ((110.8156 -7.57601 0, 110.81461 -7....  8288      8288       33   
1  POLYGON Z ((110.8147 -7.56333 0, 110.82147 -7....  8287      8287       33   
2  POLYGON Z ((110.83904 -7.55467 0, 110.83527 -7...  8286      8286       33   
3  POLYGON Z ((110.81081 -7.54494 0, 110.81097 -7...  8285      8285       33   
4  POLYGON Z ((110.83701 -7.56768 0, 110.83615 -7...  8284      8284       33   

      PROVINSI ID_KABKOT          KABKOT    ID_KEC   KECAMATAN        ID_DESA  \
0  JAWA TENGAH     33.72  KOTA SURAKARTA  33.72.02    SERENGAN  33.72.02.1004   
1  JAWA TENGAH     33.72  KOTA SURAKARTA  33.72.05  BANJARSARI  33.72.05.1007   
2  JAWA TENGAH     33.72  KOTA SURAKARTA  33.72.04      JEBRES  33.72.04.10

5️⃣ [Soal] Clip Polygon dengan Bounding Box
Deskripsi:
- Potong semua polygon dengan bounding box tertentu untuk membatasi area analisis.

Soal:

1. Buat bounding box (misal area kecil di tengah kota).
2. Potong semua polygon dengan bounding box.
3. Export hasil ke GeoJSON.

Hint:

1. Gunakan shapely.geometry.box, .intersection(), dan .to_file().

In [31]:
# 1. Buat bounding box manual (misal area kecil di tengah kota)
minx, miny, maxx, maxy = gdf.total_bounds
center_x = (minx + maxx) / 2
center_y = (miny + maxy) / 2
bbox_size = 0.02  # ~2 km

# Buat GeoJSON polygon untuk bounding box
bbox_geojson = {
    "type": "Polygon",
    "coordinates": [[
        [center_x - bbox_size, center_y - bbox_size],
        [center_x + bbox_size, center_y - bbox_size],
        [center_x + bbox_size, center_y + bbox_size],
        [center_x - bbox_size, center_y + bbox_size],
        [center_x - bbox_size, center_y - bbox_size]
    ]]
}

# Convert GeoJSON ke Shapely geometry menggunakan shape()
bbox = shape(bbox_geojson)
print(f"Bounding Box: {bbox.bounds}")
print(f"Total bounds GeoDataFrame: {gdf.total_bounds}")

# 2. Potong semua polygon dengan bounding box
gdf_clipped = gdf.copy()
gdf_clipped['geometry'] = gdf.geometry.intersection(bbox)

# Filter hanya polygon yang memiliki area > 0 (berpotongan dengan bbox)
gdf_clipped = gdf_clipped[gdf_clipped.geometry.area > 0]
print(f"Clipping selesai. {len(gdf_clipped)} polygon berpotongan dengan bounding box")

# 3. Export hasil ke GeoJSON
output_path = "ar_pop_by_zone_clipped.geojson"
gdf_clipped.to_file(output_path, driver='GeoJSON')
print(f"File tersimpan: {output_path}")

# 4. Tampilkan preview data clipped
print(f"\nPreview Clipped Polygon:")
print(gdf_clipped.head())


Bounding Box: (110.79909882650007, -7.57863222399995, 110.83909882650006, -7.538632223999951)
Total bounds GeoDataFrame: [110.76915933  -7.59522204 110.86903832  -7.52204241]
Clipping selesai. 42 polygon berpotongan dengan bounding box
File tersimpan: ar_pop_by_zone_clipped.geojson

Preview Clipped Polygon:
                                            geometry   fid  OBJECTID  ID_PROV  \
0  POLYGON Z ((110.81559 -7.57603 0, 110.81557 -7...  8288      8288       33   
1  POLYGON Z ((110.81849 -7.56487 0, 110.8185 -7....  8287      8287       33   
2  POLYGON Z ((110.83654 -7.56111 0, 110.83653 -7...  8286      8286       33   
3  POLYGON Z ((110.81081 -7.54495 0, 110.81082 -7...  8285      8285       33   
4  POLYGON Z ((110.837 -7.5677 0, 110.837 -7.5677...  8284      8284       33   

      PROVINSI ID_KABKOT          KABKOT    ID_KEC   KECAMATAN        ID_DESA  \
0  JAWA TENGAH     33.72  KOTA SURAKARTA  33.72.02    SERENGAN  33.72.02.1004   
1  JAWA TENGAH     33.72  KOTA SURAKARTA  

/var/folders/p0/dds400fd5ys_739l634yb7ch0000gn/T/ipykernel_14791/3575774104.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_clipped = gdf_clipped[gdf_clipped.geometry.area > 0]


6️⃣ [Soal] Dissolve Polygon Berdasarkan KELAS
Deskripsi:
- Gabungkan semua polygon berdasarkan atribut KELAS sehingga tiap kelas hanya punya satu geometry gabungan.

Soal:

1. Dissolve per KELAS.
2. Export hasil ke GeoJSON.

Hint:

1. Gunakan .dissolve(by='KELAS') dan .to_file().

In [20]:
# 1. Cek kolom yang tersedia
print("=== Kolom yang tersedia ===")
print(gdf.columns.tolist())

# 2. Gunakan KECAMATAN sebagai pengganti KELAS (atau gunakan kolom lain yang sesuai)
print("\n=== Nilai KECAMATAN yang ada ===")
print(gdf['KECAMATAN'].unique())
print(f"Jumlah KECAMATAN: {gdf['KECAMATAN'].nunique()}")

# 3. Dissolve per KECAMATAN
gdf_dissolved = gdf.dissolve(by='KECAMATAN')
print(f"\nDissolve per KELAS selesai. Hasil: {len(gdf_dissolved)} polygon gabungan")

# 3. Export hasil ke GeoJSON
output_path = "ar_pop_by_zone_dissolved.geojson"
gdf_dissolved.to_file(output_path, driver='GeoJSON')
print(f"File tersimpan: {output_path}")

# 4. Tampilkan info dan preview data dissolved
print(f"\n=== Info GeoDataFrame Dissolved ===")
print(f"Shape: {gdf_dissolved.shape}")
print(f"Kolom: {gdf_dissolved.columns.tolist()}")
print(f"\nPreview Dissolved Polygon:")
print(gdf_dissolved.head())


=== Kolom yang tersedia ===
['geometry', 'fid', 'OBJECTID', 'ID_PROV', 'PROVINSI', 'ID_KABKOT', 'KABKOT', 'ID_KEC', 'KECAMATAN', 'ID_DESA', 'DESA', 'JUMLAH PENDUDUK', 'BELUM BEKERJA', 'SUDAH BEKERJA', 'TIDAK ATAU BELUM SEKOLAH', 'SD dan SMP', 'SLTA', 'D1 - D3', 'S1', 'S2 >', 'PDRB 2023', 'SKOR PDRB', 'SKOR PENDIDIKAN', 'SKOR PEKERJAAN', 'SKOR PRESENTASI RUMAH TANGGA TANPA LISTRIK', 'SKOR TOTAL', 'BOBOT AKHIR', 'SOCIOECONOMIC STATUS']

=== Nilai KECAMATAN yang ada ===
<StringArray>
['SERENGAN', 'BANJARSARI', 'JEBRES', 'LAWEYAN', 'PASAR KLIWON']
Length: 5, dtype: str
Jumlah KECAMATAN: 5

Dissolve per KELAS selesai. Hasil: 5 polygon gabungan
File tersimpan: ar_pop_by_zone_dissolved.geojson

=== Info GeoDataFrame Dissolved ===
Shape: (5, 27)
Kolom: ['geometry', 'fid', 'OBJECTID', 'ID_PROV', 'PROVINSI', 'ID_KABKOT', 'KABKOT', 'ID_KEC', 'ID_DESA', 'DESA', 'JUMLAH PENDUDUK', 'BELUM BEKERJA', 'SUDAH BEKERJA', 'TIDAK ATAU BELUM SEKOLAH', 'SD dan SMP', 'SLTA', 'D1 - D3', 'S1', 'S2 >', 'PDRB 2023

7️⃣ [Soal] Union Seluruh Polygon
Deskripsi:
- Gabungkan semua polygon menjadi satu (union) tanpa atribut.

Soal:

1. Union seluruh geometry.
2. Export hasil ke GeoJSON.

Hint:

1. Gunakan shapely.ops.unary_union dan .to_file().

In [21]:
from shapely.ops import unary_union

# 1. Union seluruh geometry menjadi satu polygon
merged_geom = unary_union(gdf.geometry)
print(f"Union selesai. Tipe geometry hasil: {merged_geom.geom_type}")

# 2. Buat GeoDataFrame baru dari union geometry
gdf_union = gpd.GeoDataFrame({'geometry': [merged_geom]}, crs=gdf.crs)
print(f"GeoDataFrame union dibuat dengan CRS: {gdf_union.crs}")

# 3. Export hasil ke GeoJSON
output_path = "ar_pop_by_zone_union.geojson"
gdf_union.to_file(output_path, driver='GeoJSON')
print(f"File tersimpan: {output_path}")

# 4. Tampilkan info dan preview
print(f"\n=== Info Union Polygon ===")
print(f"Shape: {gdf_union.shape}")
print(f"Geometry type: {gdf_union.geometry.geom_type[0]}")
print(f"\nPreview Union Polygon:")
print(gdf_union)


Union selesai. Tipe geometry hasil: Polygon
GeoDataFrame union dibuat dengan CRS: EPSG:4326
File tersimpan: ar_pop_by_zone_union.geojson

=== Info Union Polygon ===
Shape: (1, 1)
Geometry type: Polygon

Preview Union Polygon:
                                            geometry
0  POLYGON Z ((110.81733 -7.58857 0, 110.81674 -7...


8️⃣ [Soal] Edit Atribut: Tambah Kolom Luas
Deskripsi:
- Tambahkan kolom luas (km²) pada GeoDataFrame.

Soal:

1. Hitung luas polygon (meter persegi), konversi ke km².
2. Export hasil ke GeoJSON.

Hint:

1. Gunakan .area dan .to_file().

In [24]:
# 1. Buat copy GeoDataFrame untuk editing atribut
gdf_with_area = gdf.copy()

# 2. Reproject ke EPSG:3857 (meter) untuk perhitungan luas yang akurat
gdf_proj = gdf_with_area.to_crs(epsg=3857)

# 3. Hitung luas dalam meter persegi, konversi ke km²
# 1 km² = 1,000,000 m²
gdf_proj['LUAS_KM2'] = gdf_proj.geometry.area / 1_000_000

# 4. Kembalikan ke EPSG:4326
gdf_with_area = gdf_proj.to_crs(epsg=4326)

print(f"Kolom LUAS_KM2 ditambahkan")
print(f"Jumlah polygon: {len(gdf_with_area)}")

# 5. Tampilkan statistik luas
print(f"\n=== Statistik Luas Polygon ===")
print(f"Luas Minimum: {gdf_with_area['LUAS_KM2'].min():.4f} km²")
print(f"Luas Maximum: {gdf_with_area['LUAS_KM2'].max():.4f} km²")
print(f"Luas Total: {gdf_with_area['LUAS_KM2'].sum():.4f} km²")
print(f"Luas Rata-rata: {gdf_with_area['LUAS_KM2'].mean():.4f} km²")

# 6. Export hasil ke GeoJSON
output_path = "ar_pop_by_zone_with_area.geojson"
gdf_with_area.to_file(output_path, driver='GeoJSON')
print(f"\nFile tersimpan: {output_path}")

# 7. Tampilkan preview data dengan kolom yang tersedia
print(f"\nPreview dengan Kolom Luas:")
print(gdf_with_area[['LUAS_KM2']].head())


Kolom LUAS_KM2 ditambahkan
Jumlah polygon: 54

=== Statistik Luas Polygon ===
Luas Minimum: 0.1942 km²
Luas Maximum: 5.7157 km²
Luas Total: 47.8502 km²
Luas Rata-rata: 0.8861 km²

File tersimpan: ar_pop_by_zone_with_area.geojson

Preview dengan Kolom Luas:
   LUAS_KM2
0  0.649836
1  0.351938
2  0.407512
3  1.502711
4  0.243643


9️⃣ [Soal] Export Subset Berdasarkan Filter
Deskripsi:
- Export subset data (misal: hanya kecamatan tertentu) ke file GeoJSON.

Soal:

1. Filter data berdasarkan KECAMATAN.
2. Export hasil ke GeoJSON.

Hint:

1. Gunakan DataFrame filtering dan .to_file().

In [25]:
# 1. Cek nilai KECAMATAN yang tersedia
print("=== Nilai KECAMATAN yang tersedia ===")
kecamatan_list = gdf['KECAMATAN'].unique()
print(kecamatan_list)
print(f"Total KECAMATAN: {len(kecamatan_list)}")

# 2. Filter data berdasarkan KECAMATAN (ambil satu atau beberapa KECAMATAN)
kecamatan_filter = kecamatan_list[0]  # Ambil KECAMATAN pertama
print(f"\nFilter berdasarkan KECAMATAN: {kecamatan_filter}")

gdf_filtered = gdf[gdf['KECAMATAN'] == kecamatan_filter]
print(f"Jumlah polygon setelah filter: {len(gdf_filtered)}")

# 3. Export hasil filter ke GeoJSON
output_path = f"subset_{kecamatan_filter.lower().replace(' ', '_')}.geojson"
gdf_filtered.to_file(output_path, driver='GeoJSON')
print(f"File tersimpan: {output_path}")

# 4. Tampilkan preview data yang di-filter
print(f"\nPreview Subset Data ({kecamatan_filter}):")
print(gdf_filtered.head())

# 5. Contoh: Filter multiple KECAMATAN (jika ingin)
print(f"\n=== Contoh: Filter Multiple KECAMATAN ===")
if len(kecamatan_list) > 1:
    kecamatan_multiple = kecamatan_list[:2]  # Ambil 2 KECAMATAN pertama
    gdf_multiple = gdf[gdf['KECAMATAN'].isin(kecamatan_multiple)]
    output_path_multiple = "subset_multiple_kecamatan.geojson"
    gdf_multiple.to_file(output_path_multiple, driver='GeoJSON')
    print(f"Filter: {kecamatan_multiple.tolist()}")
    print(f"Jumlah polygon: {len(gdf_multiple)}")
    print(f"File tersimpan: {output_path_multiple}")


=== Nilai KECAMATAN yang tersedia ===
<StringArray>
['SERENGAN', 'BANJARSARI', 'JEBRES', 'LAWEYAN', 'PASAR KLIWON']
Length: 5, dtype: str
Total KECAMATAN: 5

Filter berdasarkan KECAMATAN: SERENGAN
Jumlah polygon setelah filter: 7
File tersimpan: subset_serengan.geojson

Preview Subset Data (SERENGAN):
                                             geometry   fid  OBJECTID  \
0   MULTIPOLYGON Z (((110.8156 -7.57601 0, 110.815...  8288      8288   
9   MULTIPOLYGON Z (((110.82044 -7.58212 0, 110.82...  8279      8279   
26  MULTIPOLYGON Z (((110.82177 -7.57816 0, 110.82...  8262      8262   
33  MULTIPOLYGON Z (((110.82192 -7.57019 0, 110.82...  8255      8255   
38  MULTIPOLYGON Z (((110.82284 -7.59143 0, 110.82...  8250      8250   

    ID_PROV     PROVINSI ID_KABKOT          KABKOT    ID_KEC KECAMATAN  \
0        33  JAWA TENGAH     33.72  KOTA SURAKARTA  33.72.02  SERENGAN   
9        33  JAWA TENGAH     33.72  KOTA SURAKARTA  33.72.02  SERENGAN   
26       33  JAWA TENGAH     33.72  